In [26]:
import pandas as pd
import pyproj
from shapely.geometry import shape
from shapely.ops import transform
import ast
# Load JSON file
data_treenation = pd.read_json('/home/idisc02/Saarland_Forest_monitoring_research/Forest_Monitoring/TN/treenation_new.json')

# Convert to CSV
data_treenation.to_csv('Tree_Nation.csv', index=False)

In [27]:
import pandas as pd
df=pd.read_csv("Tree_Nation.csv",encoding='latin-1')
df.head()

,address,created_at,description,id,image,name,project_id,slug,TreeCount,polygon_data
0,,2007-05-01 12:09:33+00:00,La plantation de Dosso est la plantation princ...,1,https://treenation-uploads.s3.eu-central-1.ama...,Dosso Centre forestier,1,dosso-centre-forestier,289959,NaN
1,,2008-02-01 13:10:20+00:00,PÃ©piniÃ¨re situÃ©e dans une zone aride. Cette...,2,https://treenation-uploads.s3.eu-central-1.ama...,Bonkoukou,1,bonkoukou,0,NaN
2,,2008-02-01 14:11:52+00:00,"Dans la commune rurale de Kourfeye centre, can...",4,https://treenation-uploads.s3.eu-central-1.ama...,Talakia,1,talakia,53700,"[[14.053844216867532,3.3945930004119873],[14.0..."
3,,2007-03-01 14:35:21+00:00,Notre premiÃ¨re pÃ©piniÃ¨re situÃ©e Ã l'INRAN...,5,https://treenation-uploads.s3.eu-central-1.ama...,Niamey,1,niamey,0,NaN
4,,2016-04-08 12:39:11+00:00,El objetivo es el aumento de la cantidad y cal...,10,https://treenation-uploads.s3.eu-central-1.ama...,JardÃ­n del EdÃ©n,2,jardin-del-eden,1000,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       191 non-null    object
 1   created_at    655 non-null    object
 2   description   652 non-null    object
 3   id            655 non-null    int64 
 4   image         655 non-null    object
 5   name          655 non-null    object
 6   project_id    655 non-null    int64 
 7   slug          655 non-null    object
 8   TreeCount     655 non-null    int64 
 9   polygon_data  600 non-null    object
dtypes: int64(3), object(7)
memory usage: 51.3+ KB


In [29]:

# Counting the occurrences of "[]" within 'sites' column
empty_site_count = df[df['polygon_data'] == '[]'].shape[0]

print("Number of Empty Dictionaries in 'sites' column:", empty_site_count)

Number of Empty Dictionaries in 'sites' column: 0


In [30]:
# Counting the occurrences of NaN within 'polygon_data' column
nan_polygon_data_count = df['polygon_data'].isna().sum()

print("Number of NaN values in 'polygon_data' column:", nan_polygon_data_count)

Number of NaN values in 'polygon_data' column: 55


In [31]:


# Dropping rows where 'polygon_data' is null
df = df.dropna(subset=['polygon_data'])

In [32]:
unique_id_count = df['id'].nunique()
print(f'Count of unique id values: {unique_id_count}')

Count of unique id values: 600


In [33]:
filter_values = [71, 918, 74, 75, 85, 366, 838, 885, 1150, 1132, 635, 634, 1052, 1202, 752, 738, 760, 799, 917]
df_filtered = df[~df['id'].isin(filter_values)]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 581 entries, 2 to 654
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       171 non-null    object
 1   created_at    581 non-null    object
 2   description   578 non-null    object
 3   id            581 non-null    int64 
 4   image         581 non-null    object
 5   name          581 non-null    object
 6   project_id    581 non-null    int64 
 7   slug          581 non-null    object
 8   TreeCount     581 non-null    int64 
 9   polygon_data  581 non-null    object
dtypes: int64(3), object(7)
memory usage: 49.9+ KB


In [34]:
filter_values = [71, 918, 74, 75, 85, 366, 838, 885, 1150, 1132, 635, 634, 1052, 1202, 752, 738, 760, 799, 917]
df_duplicated= df[df['id'].isin(filter_values)]
df_duplicated.head()

,address,created_at,description,id,image,name,project_id,slug,TreeCount,polygon_data
108,NaN,2016-10-17 13:53:18+00:00,Forest Reserve that is degraded and covered wi...,71,https://treenation-uploads.s3.eu-central-1.ama...,Lower Imenti Forest (2017 - 2019),31,lower-imenti-forest-2017-2019,239364,"[[0.14312092099234122,37.662557942190496],[0.1..."
120,NaN,2016-11-04 14:49:11+00:00,The Eastern Arc Mountains are a global biodive...,74,https://treenation-uploads.s3.eu-central-1.ama...,"Yoghoi, Mhelo & Irente (2016-2017)",41,yoghoi-mhelo-irente-2016-2017,679540,"[[-4.890147631776276,38.29878217802337],[-4.89..."
121,NaN,2016-11-09 07:43:46+00:00,The project will be implemented in the Western...,75,https://treenation-uploads.s3.eu-central-1.ama...,Mile 5D Mombo-Lushoto (2016-2017),41,mile-5d-mombo-lushoto-2016-2017,235940,"[[-4.874461386623338,38.29745376476626],[-4.87..."
122,NaN,2016-12-02 04:18:59+00:00,The Area is very much populated with populatio...,85,https://treenation-uploads.s3.eu-central-1.ama...,Mtae Lushoto Tanga Tanzania (2017),41,mtae-lushoto-tanga-tanzania-2017,178000,"[[-4.789664290527133,38.29518497091674],[-4.79..."
123,NaN,2019-06-29 14:10:28+00:00,Tree Nation and Fiend of Usambara wants to bec...,366,https://treenation-uploads.s3.eu-central-1.ama...,Usambara Hill Tops (2020-2021),41,usambara-hill-tops-2020-2021,382505,"[[-4.87178851885896,38.302509757932626],[-4.87..."


### Drop Polygons with less than 50  trees planted

In [35]:

df_filtered = df_filtered[df_filtered['TreeCount'] >=50]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 383 entries, 2 to 652
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       133 non-null    object
 1   created_at    383 non-null    object
 2   description   381 non-null    object
 3   id            383 non-null    int64 
 4   image         383 non-null    object
 5   name          383 non-null    object
 6   project_id    383 non-null    int64 
 7   slug          383 non-null    object
 8   TreeCount     383 non-null    int64 
 9   polygon_data  383 non-null    object
dtypes: int64(3), object(7)
memory usage: 32.9+ KB


In [36]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import ast

def convert_to_list(data):
    if isinstance(data, list):
        return data
    try:
        
        if data.endswith(","):
            data = data[:-1]
        return ast.literal_eval(data)
    except SyntaxError:
        print(f"Failed to convert string to list: {data}")
        return None


df_filtered['polygon_data'] = df_filtered['polygon_data'].apply(convert_to_list)


df_filtered = df_filtered[df_filtered['polygon_data'].notna()]


df_filtered['polygon_data'] = df_filtered['polygon_data'].apply(lambda x: Polygon(x) if len(x) >= 4 else None)


df_filtered = df_filtered[df_filtered['polygon_data'].notna()]


gdf = gpd.GeoDataFrame(df_filtered, geometry='polygon_data')

#  GeoJSON
gdf.to_file("Tree_nation_without_duplicates.geojson", driver='GeoJSON')

#shapefile
gdf.to_file("Tree_nation_without_duplicates.shp")

/tmp/ipykernel_74456/3341716676.py:37: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("Tree_nation_without_duplicates.shp")


# Geopandas Data EDA

In [50]:
import geopandas
#from geodatasets import get_path

path_to_data = "/home/idisc02/Forest_Monitoring/src/Tree_nation_without_duplicates.shp"

gdf_tn = geopandas.read_file(path_to_data)



In [51]:
gdf_tn

,address,created_at,descriptio,id,image,name,project_id,slug,TreeCount,geometry
0,None,2008-02-01 14:11:52+00:00,"Dans la commune rurale de Kourfeye centre, can...",4,https://treenation-uploads.s3.eu-central-1.ama...,Talakia,1,talakia,53700,"POLYGON ((14.05384 3.39459, 14.05600 3.39534, ..."
1,None,2016-04-08 12:47:19+00:00,El objetivo es el aumento de la cantidad y cal...,11,https://treenation-uploads.s3.eu-central-1.ama...,Palan Bilampi,2,palan-bilampi,2600,"POLYGON ((13.00644 -85.23717, 13.00606 -85.240..."
2,None,2012-08-01 12:55:25+00:00,El objetivo es el aumento de la cantidad y cal...,13,https://treenation-uploads.s3.eu-central-1.ama...,Lirio de los Valles,2,lirio-de-los-valles,3100,"POLYGON ((13.05164 -85.47683, 13.04844 -85.477..."
3,None,2016-04-08 12:58:28+00:00,El objetivo es el aumento de la cantidad y cal...,14,https://treenation-uploads.s3.eu-central-1.ama...,San Isidro- Wanawas,2,san-isidro-wanawas,2300,"POLYGON ((13.09804 -85.24369, 13.09641 -85.247..."
4,None,2016-04-22 11:52:52+00:00,"Amazonia Reforestation is a 3,000 ha (7,410 ac...",28,https://treenation-uploads.s3.eu-central-1.ama...,Amazonia Reforestation,3,amazonia-reforestation,1104208,"POLYGON ((6.09677 -67.72642, 6.07330 -67.72239..."
...,...,...,...,...,...,...,...,...,...,...
375,None,2023-04-27 12:19:00+00:00,Largest state forest in Georgia (14k ha). Focu...,1138,https://treenation-uploads.s3.eu-central-1.ama...,Dixon Memorial NF (Georgia),664,dixon-memorial-nf-georgia,50000,"POLYGON ((31.06552 -82.18722, 31.07305 -82.181..."
376,"GPS: 3Â°21'18.8""S 39Â°58'47.6""E",2023-11-08 16:53:18+00:00,The Mida Creek mangrove reforestation site lie...,1189,https://treenation-uploads.s3.eu-central-1.ama...,Mida Creek,704,mida-creek1,471999,"POLYGON ((-3.37617 39.97235, -3.37744 39.97278..."
377,"GPS: 2Â°30'29.2""S 40Â°29'47.1""E",2023-11-08 16:55:49+00:00,The Tana Delta is a unique ecosystem providing...,1190,https://treenation-uploads.s3.eu-central-1.ama...,Tana Delta,704,tana-delta1,610000,"POLYGON ((-2.64570 40.26694, -2.64512 40.27009..."
378,None,2023-11-08 16:57:58+00:00,Kiongwe's mangrove forest has experienced sign...,1191,https://treenation-uploads.s3.eu-central-1.ama...,Kiongwe,704,kiongwe,122295,"POLYGON ((-2.32413 40.76201, -2.32536 40.76224..."


In [52]:
#Total Numbers of trees Planted
total_trees = gdf_tn["TreeCount"].sum()
total_trees

44703691

In [53]:
gdf_tn.head()

,address,created_at,descriptio,id,image,name,project_id,slug,TreeCount,geometry
0,None,2008-02-01 14:11:52+00:00,"Dans la commune rurale de Kourfeye centre, can...",4,https://treenation-uploads.s3.eu-central-1.ama...,Talakia,1,talakia,53700,"POLYGON ((14.05384 3.39459, 14.05600 3.39534, ..."
1,None,2016-04-08 12:47:19+00:00,El objetivo es el aumento de la cantidad y cal...,11,https://treenation-uploads.s3.eu-central-1.ama...,Palan Bilampi,2,palan-bilampi,2600,"POLYGON ((13.00644 -85.23717, 13.00606 -85.240..."
2,None,2012-08-01 12:55:25+00:00,El objetivo es el aumento de la cantidad y cal...,13,https://treenation-uploads.s3.eu-central-1.ama...,Lirio de los Valles,2,lirio-de-los-valles,3100,"POLYGON ((13.05164 -85.47683, 13.04844 -85.477..."
3,None,2016-04-08 12:58:28+00:00,El objetivo es el aumento de la cantidad y cal...,14,https://treenation-uploads.s3.eu-central-1.ama...,San Isidro- Wanawas,2,san-isidro-wanawas,2300,"POLYGON ((13.09804 -85.24369, 13.09641 -85.247..."
4,None,2016-04-22 11:52:52+00:00,"Amazonia Reforestation is a 3,000 ha (7,410 ac...",28,https://treenation-uploads.s3.eu-central-1.ama...,Amazonia Reforestation,3,amazonia-reforestation,1104208,"POLYGON ((6.09677 -67.72642, 6.07330 -67.72239..."


In [54]:
# Import necessary library
import geopandas as gpd
gdf_tn = gdf_tn.set_crs(epsg=4326)

# Ensure the GeoDataFrame is in a CRS that has metric units, if not convert it
if gdf_tn.crs.is_geographic:
    gdf_tn = gdf_tn.to_crs(epsg=3857)  # EPSG:3857 is a common CRS in meters

# Calculate area in square meters and convert to square kilometers
gdf_tn['site_sqkm'] = gdf_tn['geometry'].area / 10**6

/home/idisc02/anaconda3/envs/downgrade/lib/python3.8/site-packages/shapely/measurement.py:44: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


In [55]:
total_area_kmsq= gdf_tn["site_sqkm"].sum()
total_area_kmsq

87736.08065918915

In [56]:

column_name_mapping = {
    'created_at': 'planting_date_reported',
    'descriptio': 'description_reported',
    'id': 'site_id_reported',
    'geometry': 'geometry_reported',
    'project_id': 'project_id_reported',
    'TreeCount':'trees_planted_reported'
    
 
}


gdf_tn.rename(columns=column_name_mapping, inplace=True)

In [57]:
column_name = {'planting_date_reported','description_reported','site_id_reported','geometry_reported',
    'project_id_reported','trees_planted_reported',"site_sqkm"
    
 
}

In [58]:

columns_to_keep = list(column_name)

filtered_df = gdf_tn[columns_to_keep]

filtered_df.head()

,site_sqkm,site_id_reported,trees_planted_reported,geometry_reported,planting_date_reported,description_reported,project_id_reported
0,0.158513,4,53700,"POLYGON ((1564466.782 378105.633, 1564706.896 ...",2008-02-01 14:11:52+00:00,"Dans la commune rurale de Kourfeye centre, can...",1
1,1.267030,11,2600,"POLYGON ((1447870.211 -20282194.303, 1447828.3...",2016-04-08 12:47:19+00:00,El objetivo es el aumento de la cantidad y cal...,2
2,3.509979,13,3100,"POLYGON ((1452901.538 -20611852.435, 1452545.5...",2012-08-01 12:55:25+00:00,El objetivo es el aumento de la cantidad y cal...,2
3,13.353133,14,2300,"POLYGON ((1458066.883 -20290945.794, 1457885.4...",2016-04-08 12:58:28+00:00,El objetivo es el aumento de la cantidad y cal...,2
4,13.388082,28,1104208,"POLYGON ((678689.830 -10366176.004, 676077.111...",2016-04-22 11:52:52+00:00,"Amazonia Reforestation is a 3,000 ha (7,410 ac...",3


In [59]:
filtered_df[['survival_rate_reported', 'species_count_reported', 'species_planted_reported']] = None
filtered_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   site_sqkm                 327 non-null    float64 
 1   site_id_reported          380 non-null    int64   
 2   trees_planted_reported    380 non-null    int64   
 3   geometry_reported         380 non-null    geometry
 4   planting_date_reported    380 non-null    object  
 5   description_reported      378 non-null    object  
 6   project_id_reported       380 non-null    int64   
 7   survival_rate_reported    0 non-null      object  
 8   species_count_reported    0 non-null      object  
 9   species_planted_reported  0 non-null      object  
dtypes: float64(1), geometry(1), int64(3), object(5)
memory usage: 29.8+ KB


/home/idisc02/anaconda3/envs/downgrade/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/idisc02/anaconda3/envs/downgrade/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/idisc02/anaconda3/envs/downgrade/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a c